Imports & Declaraciones

In [ ]:
import numpy as np
import pandas as pd
import os
import glob
import requests

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

import warnings
warnings.filterwarnings(action='ignore')

Carga de datos

In [ ]:
# Primera parte donde se unen los csv
os.chdir("../Charts")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

df = pd.concat([pd.read_csv(f).drop(0) for f in all_filenames ],ignore_index=True)
headers =  ["Position","Track Name","Artist","Streams","URL"]
df.columns = headers

# Creamos una columna de si una cancion es hit o no (1- Hit | 0- No Hit)
df['es_hit'] = 0

# Agregar hit a las primeras 50 filas (canciones de cada semana)
for i in range(0, len(df),200):
    for j in range(i,i+50):
        df.at[j, 'es_hit'] = 1

# Eliminamos los duplicados: En caso de que una canción repetida sea hit en un caso y en otro no, se considera hit
df = df.sort_values(by='Position', ascending=True)
df = df.drop_duplicates(subset='URL', keep="first")

#Desordenamos las filas y reseteamos los indices
df = df.sample(frac=1.0, random_state=1).reset_index(drop=True) 

Agregamos features de la API de Spotify

In [ ]:
# Me quedo con el id de la cancion
df['URL'] = df['URL'].str[31:]

# SPOTIFY

CLIENT_ID  = "*****************************"
CLIENT_SECRET  = "****************************"

AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

# base URL of all Spotify API endpoints
BASE_URL = 'https://api.spotify.com/v1/'

df_features = pd.DataFrame()

# Reinicio indices para hacer que coincidan
df.reset_index(drop=True, inplace=True) 

# GET request a partir del ID de la canción
for index, row in df.iterrows():
    r = requests.get(BASE_URL + 'audio-features/' + row['URL'], headers=headers)
    r = r.json()
    #  Creamos un Dataframe con el json
    df_json = pd.json_normalize(r)
    # Añadimos la fila al Dataframe de features de Spotify
    df_features = df_features.append(df_json, ignore_index=True)

# Agregamos los nuevos features al dataframe original
df = pd.concat([df, df_features], axis=1)

# Guardamos el Dataframe en csv
df.to_csv('datos_spotify.csv', index=False)

Leemos el CSV (no usamos el generado anteriormente, ya que cuando nos traemos los datos de la API desde COLAB nos genera muchos valores NaN, a diferencia de cuando lo hacemos en PyCharm.

In [ ]:
df = pd.read_csv('./datos_spotify.csv')
df = df.sample(frac=1.0, random_state=1).reset_index(drop=True) #Desordena filas y reseteamos los indices

Modelos de training y medidas

In [ ]:
layers = (6,6)  # Entrada + capa oculta
modelos = {
    "                            Regresión logística": LogisticRegression(random_state=1),
    "                            K-Nearest Neighbors": KNeighborsClassifier(n_neighbors=4),
    "                              Árbol de decisión": DecisionTreeClassifier(random_state=1),
    "Máquinas de vectores de soporte (Linear Kernel)": LinearSVC(C=1.2,random_state=1),
    "   Máquinas de vectores de soporte (RBF Kernel)": SVC(C=1.7,random_state=1),
    "                                   Red Neuronal": MLPClassifier(solver='sgd',
                        activation='logistic',
                        alpha=1e-3,
                        hidden_layer_sizes=layers,  # Neuronas en la capa de entrada y cada capa oculta
                        random_state=1,
                        max_iter=2000),
    "                               Bosque aleatorio": RandomForestClassifier(n_estimators=500,random_state=1)
}

# Medidas
medidas = []
for i,_ in enumerate(modelos):
  medidas.append({ 'accuracy': [], 'recall': [], 'f1': [] })

Manipulacion de los datos

In [ ]:
# Eliminamos los features descriptivos -> No aportan nada al dataset
df = df.drop(['Position', 'Streams', 'Track Name', 'URL', 'id','track_href','analysis_url','type','uri'], axis=1)

# Número de FOLDS
Nfolds = 5

y = df['es_hit']
X = df.drop('es_hit', axis=1)

# Codificación One Hot para los artistas
X = pd.concat([X, pd.get_dummies(X['Artist'])], axis=1)
X = X.drop(['Artist'],axis=1)

y = y.to_numpy();
X = X.to_numpy();

Validación cruzada

In [ ]:
skf = StratifiedKFold(n_splits=Nfolds, random_state=1)  # KFold estratificado

for i,(train_index, test_index) in enumerate(skf.split(X,y)):
    
    print('\n\nProcesando partición {}/{}'.format(i+1,Nfolds))

    #-----------------------------------
    # Asignación de datos y etiquetas
    # segun los índices del fold
    #-----------------------------------
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    #------------------------------------------------
    # Estandarización de los datos
    # (sobre cada fold)
    #------------------------------------------------
    scaler = StandardScaler()
    
    #------------------------------------------
    # Ajuste del scaler con los datos de train
    #------------------------------------------
    scaler.fit(X_train)
    # Transformación de datos train y test
    #------------------------------------------
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    #Entrenamiento
    for nombre_modelo, modelo in modelos.items():
      modelo.fit(X_train, y_train)
      print(nombre_modelo + " entrenado.")

    #Testeo
    j=0
    for nombre_modelo, modelo in modelos.items():
      y_pred = modelo.predict(X_test)

      # Medidas de desempeño
      _, recall, f1,_ = precision_recall_fscore_support(y_test, y_pred, average='macro')
      accuracy = accuracy_score(y_test, y_pred)

      # Almacenamiento de las medidas
      medidas[j]['accuracy'].append(accuracy)
      medidas[j]['recall'].append(recall)
      medidas[j]['f1'].append(f1)
      j = j+1

Resultados de la validación cruzada

In [ ]:
print('---                       k-Ffold CV                     ---')
for i,nombre_modelo in enumerate(modelos):
  print('---     Modelo:' + nombre_modelo + '     ---')
  print('Fold  Acc   Recall   F1')
  for fold in range(0,Nfolds):
    print('{:3d} '.format(fold), \
          '{:.4f}'.format(medidas[i]['accuracy'][fold]),\
          '{:.4f}'.format(medidas[i]['recall'][fold]),\
          '{:.4f}'.format(medidas[i]['f1'][fold]))
    
  # Promedio de las medidas
  print('Prom', \
      '{:.4f}'.format(np.mean(medidas[i]['accuracy'])), \
      '{:.4f}'.format(np.mean(medidas[i]['recall'])), \
      '{:.4f}'.format(np.mean(medidas[i]['f1'])))